In [1]:
%load_ext blackcellmagic

In [2]:
import pandas as pd

In [64]:
prices = pd.read_csv("tmp/prices.csv", index_col=0)
prices.date = pd.to_datetime(prices.date)
prices.instrument = prices.instrument.astype(str)
prices["month_begin"] = pd.to_datetime(prices.date) - pd.offsets.MonthBegin(0)
prices = prices.set_index(["date", "instrument"])
p = prices.groupby(level=0).price
for t in range(4):
    prices[f"price_t-{t}M"] = p.shift(-t)
for t in range(1, 4):
    prices[f"price_t-{t}M_d1"] =  (prices[f"price_t-{t}M"] - prices[f"price_t-{t-1}M"]) / prices[f"price_t-{t-1}M"]
prices = prices.drop(columns=["price"])
prices = prices.reset_index()
prices = prices[
    prices.instrument.isin(
        prices.instrument.unique()[
            (
                prices.groupby("instrument").max().date
                - prices.groupby("instrument").min().date
            ).dt.days
            > 365
        ]
    )
]
prices.head()

,date,instrument,month_begin,price_t-0M,price_t-1M,price_t-2M,price_t-3M,price_t-1M_d1,price_t-2M_d1,price_t-3M_d1
0,2014-11-30,101116,2014-12-01,2281.200000,1127.975087,862.519686,413.400000,-0.505534,-0.235338,-0.520707
1,2014-11-30,1080688,2014-12-01,1127.975087,862.519686,413.400000,343700.000000,-0.235338,-0.520707,830.398162
2,2014-11-30,1080928,2014-12-01,862.519686,413.400000,343700.000000,36212.932330,-0.520707,830.398162,-0.894638
3,2014-11-30,1082585,2014-12-01,413.400000,343700.000000,36212.932330,431.800000,830.398162,-0.894638,-0.988076
4,2014-11-30,1083443,2014-12-01,343700.000000,36212.932330,431.800000,3232.686707,-0.894638,-0.988076,6.486537


In [4]:
indices = pd.read_csv("tmp/indices.csv", index_col=0)
indices = indices[indices.index_name == "תא 35"]
indices.instrument = indices.instrument.astype(str)
indices.weight = indices.weight / 100
indices.date = pd.to_datetime(indices.date)
indices["month_begin"] = pd.to_datetime(indices.date) - pd.offsets.MonthBegin(0)
indices.head()

,date,index_name,instrument,weight,month_begin
0,2017-11-30,תא 35,1097260,0.004067,2017-12-01
1,2017-11-30,תא 35,1101534,0.007323,2017-12-01
2,2017-11-30,תא 35,1084128,0.007776,2017-12-01
3,2017-11-30,תא 35,576017,0.009422,2017-12-01
4,2017-11-30,תא 35,1083484,0.009641,2017-12-01


In [5]:
funds = pd.read_csv("tmp/dataset.csv", index_col=0)
funds.date = pd.to_datetime(funds.date)
funds["month_begin"] = pd.to_datetime(funds.date) - pd.offsets.MonthBegin(0)
funds = funds[funds.instrument.isin(indices.instrument)].reset_index(drop=True)
funds.head()

,manager,instrument,date,value,month_begin
0,510791031,1081082,2017-11-30,11035332.0,2017-12-01
1,510791031,1081082,2017-12-28,216796.0,2018-01-01
2,510791031,1081082,2017-12-31,11230168.5,2018-01-01
3,510791031,1081082,2018-01-31,12941076.0,2018-02-01
4,510791031,1081082,2018-02-28,11818880.0,2018-03-01


In [6]:
prices = prices[(funds.date.min() <= prices.date) & (prices.date <= funds.date.max())]

In [26]:
merged_with_total = funds.merge(funds.groupby(["manager", "date"]).value.sum(), on=["manager", "date"], suffixes=["", "_total"])
assert (merged_with_total.value_total >= merged_with_total.value).all()

In [27]:
merged_with_total["weight"] = merged_with_total.value/ merged_with_total.value_total
merged_with_indices = merged_with_total.merge(indices, on=["instrument", "month_begin"], suffixes=["", "_index"])
merged_with_indices.sort_values("weight", ascending=False).head()

,manager,instrument,date,value,month_begin,value_total,weight,date_index,index_name,weight_index
2987,511303661,1081124,2018-12-27,729912.6,2019-01-01,729912.60,1.000000,2018-12-31,תא 35,0.046993
4488,510938608,273011,2019-06-27,28510297.2,2019-07-01,75570518.20,0.377267,2019-06-30,תא 35,0.067425
3680,510938608,273011,2019-03-28,25780511.4,2019-04-01,68460273.40,0.376576,2019-03-31,תא 35,0.072017
357,510938608,273011,2017-12-28,2032613.5,2018-01-01,7001852.23,0.290297,2017-12-31,תא 35,0.064765
3046,511944670,1143429,2018-12-27,371404.8,2019-01-01,1403278.23,0.264669,2018-12-31,תא 35,0.006135


In [28]:
merged_with_price = merged_with_indices.merge(prices, on=["instrument", "month_begin"], suffixes=["", "_price"])
merged_with_price.head()

,manager,instrument,date,value,month_begin,value_total,weight,date_index,index_name,weight_index,date_price,price_t-0M,price_t-1M,price_t-2M,price_t-3M,price_t-1M_d1,price_t-2M_d1,price_t-3M_d1
0,510791031,1081124,2017-11-30,9389122.2,2017-12-01,3.964690e+08,0.023682,2017-11-30,תא 35,0.043914,2017-11-30,48420.0,407.1,826.8,8063.0,-0.991592,1.030951,8.752056
1,510938608,1081124,2017-11-30,5055048.0,2017-12-01,9.646772e+07,0.052401,2017-11-30,תא 35,0.043914,2017-11-30,48420.0,407.1,826.8,8063.0,-0.991592,1.030951,8.752056
2,510954498,1081124,2017-11-30,8302577.4,2017-12-01,2.043502e+08,0.040629,2017-11-30,תא 35,0.043914,2017-11-30,48420.0,407.1,826.8,8063.0,-0.991592,1.030951,8.752056
3,511303661,1081124,2017-11-30,23941269.0,2017-12-01,5.477631e+08,0.043707,2017-11-30,תא 35,0.043914,2017-11-30,48420.0,407.1,826.8,8063.0,-0.991592,1.030951,8.752056
4,511776783,1081124,2017-11-30,583461.0,2017-12-01,1.472014e+07,0.039637,2017-11-30,תא 35,0.043914,2017-11-30,48420.0,407.1,826.8,8063.0,-0.991592,1.030951,8.752056


In [29]:
merged = merged_with_price.copy()

In [30]:
merged["w_diff"] = merged.weight - merged.weight_index

In [31]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [32]:
def format_plots():
    sns.set()
    mpl.rcParams["figure.figsize"] = (15, 10)
    mpl.rcParams["savefig.dpi"] = 150
    mpl.rcParams["savefig.bbox"] = "tight"
    mpl.rcParams["figure.titlesize"] = "xx-large"
    mpl.rcParams["figure.titleweight"] = "bold"
    mpl.rcParams["axes.titlesize"] = "x-large"
    mpl.rcParams["axes.titlepad"] = 25
    mpl.rcParams["axes.titleweight"] = "bold"
    mpl.rcParams["axes.labelsize"] = "x-large"
    mpl.rcParams["xtick.labelsize"] = "large"
    mpl.rcParams["ytick.labelsize"] = "large"
    mpl.rcParams["legend.fontsize"] = "large"
    # set font
    mpl.rcParams['font.family'] = "STKaiti"
    mpl.rcParams['axes.unicode_minus'] = False  # in case minus sign is shown as box

In [33]:
format_plots()

In [34]:
merged.sort_values("w_diff", ascending=False).head(10)

,manager,instrument,date,value,month_begin,value_total,weight,date_index,index_name,weight_index,date_price,price_t-0M,price_t-1M,price_t-2M,price_t-3M,price_t-1M_d1,price_t-2M_d1,price_t-3M_d1,w_diff
2736,511303661,1081124,2018-12-27,729912.60,2019-01-01,729912.60,1.000000,2018-12-31,תא 35,0.046993,2018-12-31,42880.0,318.5,353.9,8583.0,-0.992572,0.111146,23.252614,0.953007
4237,510938608,273011,2019-06-27,28510297.20,2019-07-01,75570518.20,0.377267,2019-06-30,תא 35,0.067425,2019-06-30,49460.0,275.2,5692.0,1907.0,-0.994436,19.683140,-0.664968,0.309843
3429,510938608,273011,2019-03-28,25780511.40,2019-04-01,68460273.40,0.376576,2019-03-31,תא 35,0.072017,2019-03-31,44590.0,321.8,6806.0,1919.0,-0.992783,20.149782,-0.718043,0.304559
2795,511944670,1143429,2018-12-27,371404.80,2019-01-01,1403278.23,0.264669,2018-12-31,תא 35,0.006135,2018-12-31,40010.0,155.0,194.9,757.5,-0.996126,0.257419,2.886609,0.258534
327,510938608,273011,2017-12-28,2032613.50,2018-01-01,7001852.23,0.290297,2017-12-31,תא 35,0.064765,2017-12-31,32020.0,413.0,11430.0,1395.0,-0.987102,26.675545,-0.877953,0.225532
2774,511944670,1123017,2018-12-27,335889.18,2019-01-01,1403278.23,0.239360,2018-12-31,תא 35,0.014653,2018-12-31,6950.0,478.3,4911.0,1427.0,-0.931180,9.267614,-0.709428,0.224707
2842,511944670,576017,2018-12-27,335091.60,2019-01-01,1403278.23,0.238792,2018-12-31,תא 35,0.019631,2018-12-31,99250.0,17500.0,652.6,2459.0,-0.823678,-0.962709,2.768005,0.219161
4139,511944670,1095835,2019-06-27,2759130.76,2019-07-01,12663108.84,0.217887,2019-06-30,תא 35,0.019746,2019-06-30,6550.0,1734.0,1715.0,315.9,-0.735267,-0.010957,-0.815802,0.198141
2826,511944670,281014,2018-12-27,360892.65,2019-01-01,1403278.23,0.257178,2018-12-31,תא 35,0.069085,2018-12-31,2120.0,909.5,498.1,9236.0,-0.570991,-0.452336,17.542461,0.188094
4110,510938608,1081124,2019-06-27,17082801.00,2019-07-01,75570518.20,0.226051,2019-06-30,תא 35,0.052065,2019-06-30,53780.0,403.5,612.5,9945.0,-0.992497,0.517968,15.236735,0.173986


In [65]:
def cell(managers, instrument):
    fig, ax1 = plt.subplots()
    
    for m in managers:
        temp = merged[(merged.manager == m) & merged.instrument.isin([instrument])]
        ax1.plot(temp.date, temp.w_diff, label="diff - " + str(m))
    
    ax2 = ax1.twinx()
    tmp_prices = prices[prices.instrument.isin([instrument])]
    ax2.plot(tmp_prices.date, tmp_prices["price_t-0M"], label="price", color="red", linestyle="--")

    fig.legend()
    fig.tight_layout()
    plt.show()
widgets.interactive(cell, managers=widgets.SelectMultiple(options=sorted(merged.manager.unique())), instrument=sorted(merged.instrument.unique()))

interactive(children=(SelectMultiple(description='managers', options=(510791031, 510938608, 510954498, 5113036…

In [45]:
df.to_csv("tmp/dataset.csv")